In [ ]:
%matplotlib inline
import time
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, multivariate_normal
from scipy import io
import pickle

# Own code
sys.path.append("../")
from utils.data_utils import generate_dgp_tvp_var, generate_matrices, generate_contemp_matrices
from utils.tvp_models import TVPVARModel, tvp_ar_contemp, tvp_ar_non_contemp

# Suppress scientific notation in numpy
np.set_printoptions(suppress=True)

In [ ]:
def multiprocessing_run(prior, M):
    
    import time
    import os
    from multiprocessing import Pool, Array

    # They are going to be some disgusting warnings for the first iterations of the model (has to do with initialization)
    import warnings
    warnings.filterwarnings("ignore")

    np.random.seed(12345)
    n_iterations = 16

    iterations_set = np.arange(1,n_iterations+1,1)
    M = (M,)
    prior_list = [prior]#, "lasso_alternative", "horseshoe"]
    args_tuples = [x for sub_list in [x for sub_list in [[[(n, m, prior) for n in iterations_set] for m in M] for prior in prior_list] for x in sub_list] for x in sub_list]

    if __name__ == '__main__':

        pool = Pool()
        simulation_results = pool.starmap(simulation_run, args_tuples)

#### M = 3, $p_{0} = 0.40$ and $T = 200$

In [ ]:
M = 3

In [ ]:
def simulation_run(run, M, prior):
    
    start_time = time.time()
    
    # Fixed for simulation
    T = 200
    p = 1
    train = 175
    sparsity = 0.40
    
    y = np.loadtxt(f'../simulations/datasets/y_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    coefficients = np.loadtxt(f'../simulations/datasets/coefficients_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    
    y_matrix_contemp, X_matrix_contemp = generate_contemp_matrices(T, M, p, y)
    
    # Contemperanous values added 
    msfe_contemp, alpl_contemp, mt1t_full_contemp, mt1t_coeff_contemp, sigma_contemp, ar_model = tvp_ar_contemp(T, M, p, train, X_matrix_contemp, y_matrix_contemp, prior, print_status=False, iterations=100)
    msd_contemp = np.mean((mt1t_coeff_contemp - coefficients[:,1:train])**2)
    
    elapsed_time = time.time() - start_time
    
    print(f'Run: {run}, M: {M} & prior: {prior} -> MSD: {msd_contemp} | MSFE: {msfe_contemp.mean()} | ALPL: {alpl_contemp.mean()} | elapsed time: {round(elapsed_time,6)}')
    
    return [msfe_contemp, alpl_contemp, msd_contemp, M, prior]

In [ ]:
%%time
multiprocessing_run("svss",M)

In [ ]:
%%time
multiprocessing_run("lasso_alternative",M)

In [ ]:
%%time
multiprocessing_run("horseshoe",M)

#### M = 3, $p_{0} = 0.20$ and $T = 200$

In [ ]:
def simulation_run(run, M, prior):
    
    start_time = time.time()
    
    # Fixed for simulation
    T = 200
    p = 1
    train = 175
    sparsity = 0.20
    
    y = np.loadtxt(f'../simulations/datasets/y_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    coefficients = np.loadtxt(f'../simulations/datasets/coefficients_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    
    y_matrix_contemp, X_matrix_contemp = generate_contemp_matrices(T, M, p, y)
    
    # Contemperanous values added 
    msfe_contemp, alpl_contemp, mt1t_full_contemp, mt1t_coeff_contemp, sigma_contemp, ar_model = tvp_ar_contemp(T, M, p, train, X_matrix_contemp, y_matrix_contemp, prior, print_status=False, iterations=100)
    msd_contemp = np.mean((mt1t_coeff_contemp - coefficients[:,1:train])**2)
    
    elapsed_time = time.time() - start_time
    
    print(f'Run: {run}, M: {M} & prior: {prior} -> MSD: {msd_contemp} | MSFE: {msfe_contemp.mean()} | ALPL: {alpl_contemp.mean()} | elapsed time: {round(elapsed_time,6)}')
    
    return [msfe_contemp, alpl_contemp, msd_contemp, M, prior]

In [ ]:
multiprocessing_run("svss",M)

In [ ]:
multiprocessing_run("lasso_alternative",M)

In [ ]:
multiprocessing_run("horseshoe",M)

#### M = 3, $p_{0} = 0.40$ and $T = 100$

In [ ]:
def simulation_run(run, M, prior):
    
    start_time = time.time()
    
    # Fixed for simulation
    T = 100
    p = 1
    train = 75
    sparsity = 0.40
    
    y = np.loadtxt(f'../simulations/datasets/y_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    coefficients = np.loadtxt(f'../simulations/datasets/coefficients_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    
    y_matrix_contemp, X_matrix_contemp = generate_contemp_matrices(T, M, p, y)
    
    # Contemperanous values added 
    msfe_contemp, alpl_contemp, mt1t_full_contemp, mt1t_coeff_contemp, sigma_contemp, ar_model = tvp_ar_contemp(T, M, p, train, X_matrix_contemp, y_matrix_contemp, prior, print_status=False, iterations=100)
    msd_contemp = np.mean((mt1t_coeff_contemp - coefficients[:,1:train])**2)
    
    elapsed_time = time.time() - start_time
    
    print(f'Run: {run}, M: {M} & prior: {prior} -> MSD: {msd_contemp} | MSFE: {msfe_contemp.mean()} | ALPL: {alpl_contemp.mean()} | elapsed time: {round(elapsed_time,6)}')
    
    return [msfe_contemp, alpl_contemp, msd_contemp, M, prior]

In [ ]:
multiprocessing_run("svss",M)

In [ ]:
multiprocessing_run("lasso_alternative",M)

In [ ]:
multiprocessing_run("horseshoe",M)

#### M = 3, $p_{0} = 0.20$ and $T = 100$

In [ ]:
def simulation_run(run, M, prior):
    
    start_time = time.time()
    
    # Fixed for simulation
    T = 100
    p = 1
    train = 75
    sparsity = 0.20
    
    y = np.loadtxt(f'../simulations/datasets/y_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    coefficients = np.loadtxt(f'../simulations/datasets/coefficients_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    
    y_matrix_contemp, X_matrix_contemp = generate_contemp_matrices(T, M, p, y)
    
    # Contemperanous values added 
    msfe_contemp, alpl_contemp, mt1t_full_contemp, mt1t_coeff_contemp, sigma_contemp, ar_model = tvp_ar_contemp(T, M, p, train, X_matrix_contemp, y_matrix_contemp, prior, print_status=False, iterations=100)
    msd_contemp = np.mean((mt1t_coeff_contemp - coefficients[:,1:train])**2)
    
    elapsed_time = time.time() - start_time
    
    print(f'Run: {run}, M: {M} & prior: {prior} -> MSD: {msd_contemp} | MSFE: {msfe_contemp.mean()} | ALPL: {alpl_contemp.mean()} | elapsed time: {round(elapsed_time,6)}')
    
    return [msfe_contemp, alpl_contemp, msd_contemp, M, prior]

In [ ]:
multiprocessing_run("svss",M)

In [ ]:
multiprocessing_run("lasso_alternative",M)

In [ ]:
multiprocessing_run("horseshoe",M)

#### M = 7, $p_{0} = 0.40$ and $T = 200$

In [ ]:
M = 7

In [ ]:
def simulation_run(run, M, prior):
    
    start_time = time.time()
    
    # Fixed for simulation
    T = 200
    p = 1
    train = 175
    sparsity = 0.40
    
    y = np.loadtxt(f'../simulations/datasets/y_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    coefficients = np.loadtxt(f'../simulations/datasets/coefficients_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    
    y_matrix_contemp, X_matrix_contemp = generate_contemp_matrices(T, M, p, y)
    
    # Contemperanous values added 
    msfe_contemp, alpl_contemp, mt1t_full_contemp, mt1t_coeff_contemp, sigma_contemp, ar_model = tvp_ar_contemp(T, M, p, train, X_matrix_contemp, y_matrix_contemp, prior, print_status=False, iterations=100)
    msd_contemp = np.mean((mt1t_coeff_contemp - coefficients[:,1:train])**2)
    
    elapsed_time = time.time() - start_time
    
    print(f'Run: {run}, M: {M} & prior: {prior} -> MSD: {msd_contemp} | MSFE: {msfe_contemp.mean()} | ALPL: {alpl_contemp.mean()} | elapsed time: {round(elapsed_time,6)}')
    
    return [msfe_contemp, alpl_contemp, msd_contemp, M, prior]

In [ ]:
multiprocessing_run("svss",M)

In [ ]:
multiprocessing_run("lasso_alternative",M)

In [ ]:
multiprocessing_run("horseshoe",M)

#### M = 7, $p_{0} = 0.20$ and $T = 200$

In [ ]:
def simulation_run(run, M, prior):
    
    start_time = time.time()
    
    # Fixed for simulation
    T = 200
    p = 1
    train = 175
    sparsity = 0.20
    
    y = np.loadtxt(f'../simulations/datasets/y_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    coefficients = np.loadtxt(f'../simulations/datasets/coefficients_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    
    y_matrix_contemp, X_matrix_contemp = generate_contemp_matrices(T, M, p, y)
    
    # Contemperanous values added 
    msfe_contemp, alpl_contemp, mt1t_full_contemp, mt1t_coeff_contemp, sigma_contemp, ar_model = tvp_ar_contemp(T, M, p, train, X_matrix_contemp, y_matrix_contemp, prior, print_status=False, iterations=100)
    msd_contemp = np.mean((mt1t_coeff_contemp - coefficients[:,1:train])**2)
    
    elapsed_time = time.time() - start_time
    
    print(f'Run: {run}, M: {M} & prior: {prior} -> MSD: {msd_contemp} | MSFE: {msfe_contemp.mean()} | ALPL: {alpl_contemp.mean()} | elapsed time: {round(elapsed_time,6)}')
    
    return [msfe_contemp, alpl_contemp, msd_contemp, M, prior]

In [ ]:
multiprocessing_run("svss",M)

In [ ]:
multiprocessing_run("lasso_alternative",M)

In [ ]:
multiprocessing_run("horseshoe",M)

#### M = 7, $p_{0} = 0.40$ and $T = 100$

In [ ]:
def simulation_run(run, M, prior):
    
    start_time = time.time()
    
    # Fixed for simulation
    T = 100
    p = 1
    train = 75
    sparsity = 0.40
    
    y = np.loadtxt(f'../simulations/datasets/y_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    coefficients = np.loadtxt(f'../simulations/datasets/coefficients_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    
    y_matrix_contemp, X_matrix_contemp = generate_contemp_matrices(T, M, p, y)
    
    # Contemperanous values added 
    msfe_contemp, alpl_contemp, mt1t_full_contemp, mt1t_coeff_contemp, sigma_contemp, ar_model = tvp_ar_contemp(T, M, p, train, X_matrix_contemp, y_matrix_contemp, prior, print_status=False, iterations=100)
    msd_contemp = np.mean((mt1t_coeff_contemp - coefficients[:,1:train])**2)
    
    elapsed_time = time.time() - start_time
    
    print(f'Run: {run}, M: {M} & prior: {prior} -> MSD: {msd_contemp} | MSFE: {msfe_contemp.mean()} | ALPL: {alpl_contemp.mean()} | elapsed time: {round(elapsed_time,6)}')
    
    return [msfe_contemp, alpl_contemp, msd_contemp, M, prior]

In [ ]:
multiprocessing_run("svss",M)

In [ ]:
multiprocessing_run("lasso_alternative",M)

In [ ]:
multiprocessing_run("horseshoe",M)

#### M = 7, $p_{0} = 0.20$ and $T = 100$

In [ ]:
def simulation_run(run, M, prior):
    
    start_time = time.time()
    
    # Fixed for simulation
    T = 100
    p = 1
    train = 75
    sparsity = 0.20
    
    y = np.loadtxt(f'../simulations/datasets/y_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    coefficients = np.loadtxt(f'../simulations/datasets/coefficients_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    
    y_matrix_contemp, X_matrix_contemp = generate_contemp_matrices(T, M, p, y)
    
    # Contemperanous values added 
    msfe_contemp, alpl_contemp, mt1t_full_contemp, mt1t_coeff_contemp, sigma_contemp, ar_model = tvp_ar_contemp(T, M, p, train, X_matrix_contemp, y_matrix_contemp, prior, print_status=False, iterations=100)
    msd_contemp = np.mean((mt1t_coeff_contemp - coefficients[:,1:train])**2)
    
    elapsed_time = time.time() - start_time
    
    print(f'Run: {run}, M: {M} & prior: {prior} -> MSD: {msd_contemp} | MSFE: {msfe_contemp.mean()} | ALPL: {alpl_contemp.mean()} | elapsed time: {round(elapsed_time,6)}')
    
    return [msfe_contemp, alpl_contemp, msd_contemp, M, prior]

In [ ]:
multiprocessing_run("svss",M)

In [ ]:
multiprocessing_run("lasso_alternative",M)

In [ ]:
multiprocessing_run("horseshoe",M)